In [5]:
import numpy as np
import pandas as pd
import seaborn as sns

from data_analysis import download_data, download_condition_counts, calc_condition_counts, ExperimentDataLoader
from itertools import product
from frozendict import frozendict
from tqdm.notebook import tqdm
from functools import lru_cache
from construal_shifting.task_modeling.participant_model import ParticipantModel
from construal_shifting.task_modeling.construal_trial_model import ConstrualTrialModel

exp_data = ExperimentDataLoader(
    trialdata_file="rawtrialdata-anon.csv"
)

Participant participant_021:anonymized missing trials
Trial Indices = [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]; expected [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
Participant participant_096:anonymized missing trials
Trial Indices = [0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]; expected [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


# Analysis with fit model

In [6]:
model_fit_df = pd.read_pickle('./data/model_fits.pkl')
model_fit_df['df'] = model_fit_df[[
    'fit_construal_cost_weight', 'fit_construal_set_stickiness', 'fit_action_random_choice'
]].apply(lambda r: sum([t == True for t in r]), axis=1)
model_fit_df['AIC'] = model_fit_df.apply(lambda r: 2*r['df'] + r['NLL'], axis=1)
model_fit_df['dAIC'] = model_fit_df['AIC'] - model_fit_df['AIC'].min()

In [8]:
model_fits = model_fit_df.copy()
model_fits['Model'] = [
    "No complexity or switch cost",
    "Only switch cost",
    "Only complexity cost",
    "Both complexity and switch cost"
]
model_fits = model_fits[['Model', 'construal_cost_weight', 'construal_set_stickiness', 'action_random_choice', 'df', 'AIC', 'dAIC']]
model_fits = model_fits.sort_values(by="dAIC", ascending=False)
model_fits = model_fits.round(dict(
    action_random_choice=2, 
    construal_cost_weight=2,
    construal_set_stickiness=2,
))
model_fits['AIC'] = model_fits['AIC'].astype(int)
model_fits['dAIC'] = model_fits['dAIC'].astype(int)
model_fits = model_fits.rename(columns=dict(
    action_random_choice=r'$\varepsilon_{\text{Move}}$', 
    construal_cost_weight=r'$\lambda_{\text{Complexity}}$',
    construal_set_stickiness=r'$\lambda_{\text{Switch}}$',
    AIC="AIC",
    dAIC=r"$\Delta$AIC"
))
model_fits_table = model_fits.to_latex(
    # na_rep="---", 
    escape=False,
    column_format="lrrrrrr",
    # col_space=1,
    index=False
)
# model_fits
print(model_fits_table)

\begin{tabular}{lrrrrrr}
\toprule
                          Model &  $\lambda_{\text{Complexity}}$ &  $\lambda_{\text{Switch}}$ &  $\varepsilon_{\text{Move}}$ &  df &   AIC &  $\Delta$AIC \\
\midrule
   No complexity or switch cost &                           0.00 &                        0.0 &                          0.1 &   1 & 95931 &          239 \\
               Only switch cost &                           0.08 &                        0.0 &                          0.1 &   2 & 95930 &          238 \\
           Only complexity cost &                           0.00 &                       10.0 &                          0.1 &   2 & 95762 &           70 \\
Both complexity and switch cost &                           0.64 &                       10.0 &                          0.1 &   3 & 95691 &            0 \\
\bottomrule
\end{tabular}



In [10]:
model_fits = model_fit_df.copy()
model_fits['Model'] = [
    "No complexity or switch cost",
    "Only switch cost",
    "Only complexity cost",
    "Both complexity and switch cost"
]
model_fits = model_fits[['Model', 'construal_cost_weight', 'construal_set_stickiness', 'action_random_choice', 'df', 'AIC', 'dAIC']]
model_fits = model_fits.sort_values(by="dAIC", ascending=False)
model_fits.round(2)

,Model,construal_cost_weight,construal_set_stickiness,action_random_choice,df,AIC,dAIC
0,No complexity or switch cost,0.00,0.0,0.1,1,95931.32,239.43
1,Only switch cost,0.08,0.0,0.1,2,95930.34,238.45
2,Only complexity cost,0.00,10.0,0.1,2,95762.44,70.55
3,Both complexity and switch cost,0.64,10.0,0.1,3,95691.89,0.00
